In [3]:
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from transformers import AutoTokenizer
from datasets import load_dataset
from tqdm import tqdm

from instruct_goose.agent import Agent
from instruct_goose.reward import RewardModel, LitRewardModel, RewardLoss
from instruct_goose.dataset import PairDataset, PromptDataset
from instruct_goose.utils import load_yaml

In [4]:
config = load_yaml("../configs/sentiment_config.yml")

# reward
reward_checkpoint = config["reward_model"]["model_path"]
reward_data_path = config["reward_data"]["data_path"]
batch_size = config["reward_data"]["batch_size"]

In [3]:
reward_dataset = load_dataset(reward_data_path)
tokenizer = AutoTokenizer.from_pretrained(reward_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

Using custom data configuration CarperAI--openai_summarize_comparisons-e658a7d8b35ec187
Found cached dataset parquet (/Users/education/.cache/huggingface/datasets/CarperAI___parquet/CarperAI--openai_summarize_comparisons-e658a7d8b35ec187/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)
100%|██████████| 2/2 [00:00<00:00, 36.22it/s]


In [4]:
small_reward_dataset, _ = random_split(reward_dataset["train"], [10, len(reward_dataset["train"]) - 10])

In [5]:
reward_model = RewardModel(reward_checkpoint)

In [ ]:
# pair_dataset = PairDataset(reward_dataset["train"], tokenizer, max_length=1024)
pair_dataset = PairDataset(small_reward_dataset, tokenizer, max_length=1024)

100%|██████████| 10/10 [00:00<00:00, 401.83it/s]


In [ ]:
pair_dataset[0]

(tensor([[48902,    25,  3878,  ..., 50256, 50256, 50256]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0]]),
 tensor([[48902,    25,   314,  ..., 50256, 50256, 50256]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0]]))

In [ ]:
dataloader = DataLoader(pair_dataset, batch_size=2, shuffle=True)

In [ ]:
reward_loss = RewardLoss()
lit_reward = LitRewardModel(reward_model, reward_loss)

In [ ]:
trainer = pl.Trainer(max_epochs=3, log_every_n_steps=1)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/education/DATA/projects/ai/RLHF/instructGOOSE/env/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:200: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [11]:
trainer_fit = trainer.fit(lit_reward, dataloader)


  | Name      | Type        | Params
------------------------------------------
0 | model     | RewardModel | 124 M 
1 | loss_func | RewardLoss  | 0     
------------------------------------------
124 M     Trainable params
0         Non-trainable params
124 M     Total params
497.762   Total estimated model params size (MB)
/Users/education/DATA/projects/ai/RLHF/instructGOOSE/env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/education/DATA/projects/ai/RLHF/instructGOOSE/env/lib/python3.10/site-packages/pytorch_lightning/trainer/trainer.py:1595: PossibleUserWarning: The number of training batches (5) is smaller than the logging interval Trainer(log_eve

Epoch 2: 100%|██████████| 5/5 [01:28<00:00, 17.67s/it, loss=-0.248, v_num=1]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 5/5 [01:29<00:00, 17.92s/it, loss=-0.248, v_num=1]


In [9]:
for i, batch in tqdm(enumerate(dataloader)):
    chosen_input_ids, chosen_attention_mask, rejected_input_ids, rejected_attention_mask = batch
    
    chosen_rewards = reward_model(chosen_input_ids, chosen_attention_mask)
    rejected_rewards = reward_model(rejected_input_ids, rejected_attention_mask)
    
    pass

5it [01:13, 14.73s/it]


In [14]:
chosen_rewards

tensor([[2.4260, 8.0676, 6.4001,  ..., 8.4922, 6.5932, 1.5312],
        [2.3436, 7.9532, 8.3668,  ..., 9.5768, 9.3080, 8.9846]],
       grad_fn=<ReshapeAliasBackward0>)

In [21]:
chosen_rewards.shape

torch.Size([2, 1024])

In [22]:
chosen_rewards[:, -1]

tensor([1.5312, 8.9846], grad_fn=<SelectBackward0>)

In [18]:
import einops
last_token = einops.rearrange(chosen_rewards, "b n -> (b n)")

In [20]:
last_token.shape

torch.Size([2048])

In [26]:
import torch
hiddens = torch.randn([2, 1, 1024, 1])

In [27]:
hiddens.shape

torch.Size([2, 1, 1024, 1])

In [28]:
hiddens[0][0][-1]

tensor([0.0683])

### Training a language model to aligns with human preferences

In [12]:
agent_data_path = config["agent_data"]["data_path"]
agent_path = config["model"]["model_path"]
model_tokenizer_path = config["model"]["tokenizer_path"]

In [13]:
model_tokenizer_path, agent_path

('mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization',
 'mrm8488/bert-mini2bert-mini-finetuned-cnn_daily_mail-summarization')

In [ ]:
agent = Agent()

### Setup Pre-trained model

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

/Users/education/DATA/projects/ai/RLHF/instructGOOSE/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
model_checkpoint = config["model"]["model_path"]
model_tokenizer_checkpoint = config["model"]["tokenizer_path"]

In [7]:
model_checkpoint

'distilgpt2'

In [8]:
model_checkpoint

'distilgpt2'

In [12]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_tokenizer_checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [13]:
agent = Agent(model)

In [14]:
prompt = [
    "Upon once time there's a",
    "the world is going to",
    "what's up everybody"
]
inputs = tokenizer(prompt, padding=True, truncation=True, return_tensors="pt")

In [11]:
# _output = model(
#     input_ids=inputs["input_ids"],
#     attention_mask=inputs["attention_mask"],
#     output_hidden_states=True
# )

In [15]:
inputs

{'input_ids': tensor([[23792,  1752,   640,   612,   338,   257],
        [ 1169,   995,   318,  1016,   284, 50256],
        [10919,   338,   510,  7288, 50256, 50256]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 0],
        [1, 1, 1, 1, 0, 0]])}

In [16]:
logits, logprobs, entropy, value = agent(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])

In [17]:
logits.shape

torch.Size([3, 6, 50257])

In [18]:
value.shape

torch.Size([3, 1])

In [19]:
value

tensor([[-0.2941],
        [ 0.0285],
        [ 0.0207]], grad_fn=<SliceBackward0>)

In [21]:
logprobs.shape

torch.Size([3, 6, 50257])

In [29]:
entropy

tensor([[7.5797, 3.9689, 4.9965, 2.9985, 3.9015, 7.6359],
        [7.8733, 4.6695, 6.1731, 1.7047, 4.6383, 6.8685],
        [7.5955, 5.6025, 2.9470, 3.9640, 6.8450, 6.8521]],
       grad_fn=<NegBackward0>)

In [26]:
import torch
from torch.distributions import Categorical

x = torch.randn(4)
dist = Categorical(logits=x)

In [27]:
dist.entropy()

tensor(1.3791)